In [1]:
import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn import cross_validation

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
## read in VGG features extracted from last fully connected layer
VGG_FEATURE_FILE = '/notebooks/ashish/features_csv/vgg_feats.csv'
df_vgg = pd.read_csv(VGG_FEATURE_FILE, header=None)
df_vgg.rename(columns = {0: 'id', 1: 'cancer'}, inplace=True)
print ("number of rows", len(df_vgg))
df_vgg.head()

('number of rows', 1594)


,id,cancer,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,512,513
0,72a1e35c34052e163f61585ba0c9daf4,0.187917,1.177220,0.0,0.000000,0.615929,0.0,0.129078,0.000000,0.848804,...,0.0,0.0,0.0,0.414562,1.612470,0.0,0.496569,0.0,0.0,NaN
1,4b28f147cb82baba3edcdbd34ca19085,0.000000,0.373899,0.0,0.000000,0.000000,0.0,0.000000,0.164907,0.245377,...,0.0,0.0,0.0,0.201370,0.198621,0.0,0.000000,0.0,0.0,NaN
2,654f2095bace27af870cdfcc66f9717c,0.000000,0.687288,0.0,0.000000,0.000000,0.0,0.000000,0.290345,0.088511,...,0.0,0.0,0.0,0.243824,0.561189,0.0,0.249505,0.0,0.0,NaN
3,95a98df466d4f6c6689908ea9a8f324b,0.000000,0.298770,0.0,0.000000,0.000000,0.0,0.000000,0.446660,0.000000,...,0.0,0.0,0.0,0.491026,0.382245,0.0,0.118882,0.0,0.0,NaN
4,a4dc34f2731b62f60c6c390a39fe11b2,0.000000,0.410989,0.0,0.424119,0.000000,0.0,0.000000,0.621741,0.464176,...,0.0,0.0,0.0,0.785322,0.238981,0.0,0.000000,0.0,0.0,NaN


In [3]:
RESNET_FEAT_FILE = '/notebooks/ashish/features_csv/resnet_feats50.csv'
RESNET_FEAT_FILE = '/notebooks/ashish/features_csv/resnet_feats_charles.csv'
df_resnet = pd.read_csv(RESNET_FEAT_FILE)
print ("number of rows", len(df_resnet))
df_resnet.head()

('number of rows', 1595)


,0,1,10,100,1000,1001,1002,1003,1004,1005,...,991,992,993,994,995,996,997,998,999,id
0,0.555498,0.0,0.021743,0.179729,1.066932,0.521365,0.191492,1.092509,0.132298,0.006595,...,0.705790,0.607742,1.546954,0.279501,0.002673,0.413203,0.118664,0.103059,0.457758,72a1e35c34052e163f61585ba0c9daf4
1,0.208845,0.0,0.016385,0.190936,0.673486,1.353718,0.554376,1.815671,0.209664,0.424304,...,0.661181,1.148754,1.461836,0.276636,0.002274,0.713753,0.282128,0.397599,0.822301,4b28f147cb82baba3edcdbd34ca19085
2,0.314211,0.0,0.014422,0.199936,0.852416,1.699911,0.655441,1.538375,0.239300,0.456696,...,0.807780,1.181679,2.069214,0.246085,0.002106,0.608623,0.469256,0.320199,1.039385,654f2095bace27af870cdfcc66f9717c
3,0.336828,0.0,0.013074,0.332237,0.994983,1.747601,0.617796,1.724912,0.292321,0.400177,...,0.672156,1.242580,1.745696,0.189913,0.002258,0.495983,0.743609,0.220174,1.158524,95a98df466d4f6c6689908ea9a8f324b
4,0.229374,0.0,0.019242,0.237887,0.739535,2.060798,0.876296,1.194281,0.265383,0.413132,...,0.416984,1.175388,1.913326,0.089742,0.003124,0.293470,0.978236,0.266611,0.756222,a4dc34f2731b62f60c6c390a39fe11b2


In [4]:
df = pd.merge(df_vgg, df_resnet, on='id', how='left')
df.drop('cancer', axis=1, inplace=True)
print ("number of rows", len(df))
df.head()

('number of rows', 1594)


,id,2,3,4,5,6,7,8,9,10,...,990,991,992,993,994,995,996,997,998,999
0,72a1e35c34052e163f61585ba0c9daf4,1.177220,0.0,0.000000,0.615929,0.0,0.129078,0.000000,0.848804,0.445924,...,0.381874,0.705790,0.607742,1.546954,0.279501,0.002673,0.413203,0.118664,0.103059,0.457758
1,4b28f147cb82baba3edcdbd34ca19085,0.373899,0.0,0.000000,0.000000,0.0,0.000000,0.164907,0.245377,0.000000,...,0.317342,0.661181,1.148754,1.461836,0.276636,0.002274,0.713753,0.282128,0.397599,0.822301
2,654f2095bace27af870cdfcc66f9717c,0.687288,0.0,0.000000,0.000000,0.0,0.000000,0.290345,0.088511,0.000000,...,0.306902,0.807780,1.181679,2.069214,0.246085,0.002106,0.608623,0.469256,0.320199,1.039385
3,95a98df466d4f6c6689908ea9a8f324b,0.298770,0.0,0.000000,0.000000,0.0,0.000000,0.446660,0.000000,0.000000,...,0.393472,0.672156,1.242580,1.745696,0.189913,0.002258,0.495983,0.743609,0.220174,1.158524
4,a4dc34f2731b62f60c6c390a39fe11b2,0.410989,0.0,0.424119,0.000000,0.0,0.000000,0.621741,0.464176,0.776238,...,0.348067,0.416984,1.175388,1.913326,0.089742,0.003124,0.293470,0.978236,0.266611,0.756222


In [5]:
LABEL_FILE = '/notebooks/DSB3/data/stage1_labels.csv'
df_train = pd.read_csv(LABEL_FILE)
print ("df_train", len(df_train))

df_train = pd.merge(df_train, df, how='left', on='id')
print ("df_train", len(df_train))

df_train.head()

('df_train', 1397)
('df_train', 1397)


,id,cancer,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0015ceb851d7251b8f399e39779d1e7d,1,0.454916,0.0,0.0,0.000000,0.0,0.000000,0.252907,0.265739,...,0.331854,0.754011,1.048662,1.985354,0.203940,0.002858,0.627159,0.538401,0.292011,0.845529
1,0030a160d58723ff36d73f41b170ec21,0,0.379240,0.0,0.0,0.000000,0.0,0.000000,0.233654,0.268706,...,0.347848,0.528129,0.984715,2.115402,0.299098,0.003404,0.617680,0.509701,0.190050,0.975869
2,003f41c78e6acfa92430a057ac0b306e,0,0.309731,0.0,0.0,0.000000,0.0,0.000000,0.157476,0.285661,...,0.237249,0.636377,1.036543,1.678446,0.264167,0.003988,0.444304,0.820075,0.153265,1.034107
3,006b96310a37b36cccb2ab48d10b49a3,1,0.305940,0.0,0.0,0.000000,0.0,0.000000,0.215326,0.231697,...,0.256351,0.684518,1.121627,1.671193,0.206808,0.002034,0.510207,0.593624,0.165558,1.002003
4,008464bb8521d09a42985dd8add3d0d2,1,1.194530,0.0,0.0,0.224819,0.0,0.197698,0.209978,0.000000,...,0.185579,1.106468,1.116747,2.174308,0.281288,0.002268,0.658722,0.788684,0.313470,1.555531


In [6]:
feat_cols = [col for col in df_train.columns if col not in ['id', 'cancer']]
x = df_train[feat_cols].values
y = df_train['cancer'].values

In [7]:
## train xgboost model using resent and vgg net feats
print ("Training Set", x.shape, y.shape)

trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, 
                                                               y, 
                                                               random_state=2487, 
                                                               stratify=y,
                                                               test_size=0.20)
clf = xgb.XGBRegressor(max_depth=1500,
                       n_estimators=3000,
                       min_child_weight=12,
                       learning_rate=0.01,
                       nthread=8,
                       subsample=0.80,
                       colsample_bytree=0.80,
                       seed=4784)

clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=True, eval_metric='logloss', early_stopping_rounds=100)

('Training Set', (1397, 2560), (1397,))
[0]	validation_0-logloss:0.691115
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.688983
[2]	validation_0-logloss:0.6865
[3]	validation_0-logloss:0.684695
[4]	validation_0-logloss:0.682456
[5]	validation_0-logloss:0.680979
[6]	validation_0-logloss:0.679298
[7]	validation_0-logloss:0.676737
[8]	validation_0-logloss:0.674465
[9]	validation_0-logloss:0.672308
[10]	validation_0-logloss:0.670815
[11]	validation_0-logloss:0.668678
[12]	validation_0-logloss:0.66651
[13]	validation_0-logloss:0.664856
[14]	validation_0-logloss:0.663524
[15]	validation_0-logloss:0.661598
[16]	validation_0-logloss:0.65991
[17]	validation_0-logloss:0.658264
[18]	validation_0-logloss:0.656503
[19]	validation_0-logloss:0.654906
[20]	validation_0-logloss:0.653344
[21]	validation_0-logloss:0.651646
[22]	validation_0-logloss:0.650287
[23]	validation_0-logloss:0.648754
[24]	validation_0-logloss:0.647027
[25]	validation_0-logloss:0.64

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=1500,
       min_child_weight=12, missing=None, n_estimators=3000, nthread=8,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=4784, silent=True, subsample=0.8)

In [8]:
SAMPLE_SUBMISSION_FILE = '/notebooks/ashish/stage1_sample_submission.csv'
df_test_label = pd.read_csv(SAMPLE_SUBMISSION_FILE)
print ("df_test_label", len(df_test_label))

df_test = pd.merge(df_test_label, df, how='left', on='id')

print ("df_test", len(df_test))
df_test.head()

('df_test_label', 198)
('df_test', 198)


,id,cancer,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,026470d51482c93efc18b9803159c960,0.5,0.627789,0.0,0.0,0.0,0.001045,0.000000,0.442722,0.115850,...,0.314102,0.831031,1.216652,2.429907,0.254749,0.002043,0.500277,0.450879,0.305701,1.034894
1,031b7ec4fe96a3b035a8196264a8c8c3,0.5,0.597993,0.0,0.0,0.0,0.102241,0.000000,0.548600,0.020765,...,0.200911,0.728564,0.916857,1.888076,0.188965,0.002050,0.594377,0.479100,0.249953,0.778031
2,03bd22ed5858039af223c04993e9eb22,0.5,0.401532,0.0,0.0,0.0,0.000000,0.000000,0.174653,0.441126,...,0.188370,0.838596,1.302286,1.483619,0.407917,0.001000,0.933712,0.351824,0.231640,1.223717
3,06a90409e4fcea3e634748b967993531,0.5,0.552083,0.0,0.0,0.0,0.000000,0.000000,0.524672,0.125767,...,0.173932,0.593891,0.983020,1.915699,0.244136,0.001837,0.594847,0.587519,0.262463,0.991666
4,07b1defcfae5873ee1f03c90255eb170,0.5,0.493107,0.0,0.0,0.0,0.003284,0.449708,0.303647,0.000000,...,0.169441,0.781013,1.473530,1.570641,0.211159,0.004032,0.684581,0.530492,0.164307,1.251674


In [ ]:
## predict output for the test data and create submission file

OUT_SUBMISSION_FILE = '/notebooks/ashish/subm_resnet_vgg_xgboost.csv'


feat_cols = [col for col in df_test.columns if col not in ['id', 'cancer']]
x = df_test[feat_cols].values
pred = clf.predict(x)

df_test['cancer'] = pred
df_test.drop(feat_cols, axis=1, inplace=True)
df_test.to_csv(OUT_SUBMISSION_FILE, index=False)

df_test.head()